In [1]:
import os
import matplotlib.pyplot as plt
import datetime
import time
import matplotlib.gridspec as gridspec

import pickle
import numpy as np
import pandas as pd
import seaborn as sns

from itertools import compress

from helpers.expr_data import ExprData
from helpers.scale_data import ScaleData
from helpers.similarity import Similarity
from helpers.feature_selection import FeatureSelection

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import root_mean_squared_error as rmse_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

import warnings
warnings.filterwarnings("ignore")

In [2]:
SMALL_SIZE = 15
MEDIUM_SIZE = 18
BIGGER_SIZE = 22
SMALL_SMALL_SIZE = 10

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize

# plt.rc('legend', fontsize=SMALL_SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
import random
np.random.seed(42)
random.seed(42)

In [4]:
OVERALL_PLOT = False
# for measuring modeling time

In [5]:
def nrmse_score(y_true, y_pred):
    # return np.sqrt((((y_true-y_pred)/y_pred)**2).mean())    
    # return np.sqrt(((abs(y_true-y_pred)/y_pred)).mean())    
    return rmse_score(y_true, y_pred)/(np.max(y_true)-np.min(y_true))
    # return rmse_score(y_true, y_pred)/(np.mean(y_true))

score_func = make_scorer(nrmse_score, greater_is_better=False)

In [6]:
# read in the performance metrics for each experiment
data_all = ExprData()
data_all.load_pickle()
data_all = data_all.merge_tpch()

In [7]:
data = data_all.remove_by_wlname(['xml', 'ycsb'])
ycsb_data = data_all.remove_by_wlname(['xml', 'tpcc', 'tpch', 'twitter'])

In [8]:
all_groups = ['10', '11', '12']
candidate_group = '10'
ycsb_data = ycsb_data.remove_by_group([g for g in all_groups if g != candidate_group])

In [9]:
X_label = 'SKU'
expr_label = 'EXPR'
y_true_label = 'Y_TRUE'
y_pred_label = 'Y_PRED'
suffix_labels = ['_small', '_large']

In [10]:
# similarity for all
new_data = data.keep_complete_exprs()

In [11]:
sampled_data = new_data.sample_data()

In [12]:
data_by_type = sampled_data.split_by_type()

In [13]:
sampled_by_type = sampled_data.split_by_type()
for ty, expr_set in sampled_by_type.items():
    sub_by_term = expr_set.split_by_term()
    sampled_by_type[ty] = sub_by_term

In [14]:
def unroll_mtx(mtx):
    return (mtx.T).ravel()

In [15]:
def get_data_X(curr_data):
    scaler = ScaleData()
    plan_mtxs, plan_col_ranges = scaler.scale(curr_data.plan_mtxs)
    perf_mtxs, perf_col_ranges = scaler.scale(curr_data.perf_mtxs)

    simi_calc = Similarity(curr_data, plan_mtxs, plan_col_ranges, perf_mtxs, perf_col_ranges)
    simi_calc.calc_bined_mtx(plan_only=True) # all features

    keep_cols = simi_calc.filter_by_features(['MaxCompileMemory', 'CachedPlanSize', 'AvgRowSize', 'StatementSubTreeCost', 'SerialRequiredMemory', 'EstimatedPagesCached', 'CompileMemory'])
    ndarrs = [b[:, keep_cols].astype(float) for b in simi_calc.cumulative_bined]
    print(len(ndarrs))
    X = []
    for i in range(len(curr_data.cpu_nums)):
        X.append(np.insert(ndarrs[i], 0, int(curr_data.cpu_nums[i][3:])))
    # print(len(X), len(X[0]), X[0])
    return X

In [16]:
def plot_prediction(dfs, X, Y, expr_idxs, method, plot=True, figsize=(3.6,2.4)):
    # plot all trends 
    colors = ['#1b9e77','#7570b3','#d95f02','#e7298a']
    fig, ax = plt.subplots(figsize=figsize)
        
    pred_label_add = False
    pred_color = colors[0]
    overall_expr_label_add = False
    expr_tracker = 0
    
    all_true_tracker = {}
        
    color_map = [colors[1+idx%3] for idx, _ in enumerate(expr_idxs)]
    ax.scatter(X, Y, color=color_map)
    
    for df in dfs:
        expr_c = 1
        expr_label_add = False
        for _, row in df.iterrows():
            run_color = colors[1+expr_tracker%3]
            curr_x = [row[f'{X_label}{suffix}'] for suffix in suffix_labels]
            expr_tracker += 1
            true_y = [row[f'{y_true_label}{suffix}'] for suffix in suffix_labels]
            
            for x, y in zip(np.array(curr_x).flatten(), true_y):
                if x not in all_true_tracker:
                    all_true_tracker[x] = []
                all_true_tracker[x].append(y)
                
            if not overall_expr_label_add:
                if not expr_label_add and expr_tracker % 3 == 2:
                    expr = ax.plot(curr_x, true_y, '-', color=run_color, alpha=0.5, linewidth=1.5, label=f'Expr{expr_c} True_y')
                    expr_label_add = True
                else:
                    ax.plot(curr_x, true_y, '-', color=run_color, alpha=0.5, linewidth=1.5, label=f'Expr{expr_c} True_y')
            else:
                ax.plot(curr_x, true_y, '-', color=run_color, alpha=0.5, linewidth=1.5)
            expr_c += 1
        overall_expr_label_add = True
        
    for df in dfs:
        for _, row in df.iterrows():
            curr_x = [row[f'{X_label}{suffix}'] for suffix in suffix_labels]
            pred_y = [row[f'{y_true_label}{suffix_labels[0]}'], row[f'{y_pred_label}{suffix_labels[1]}']]
            
            if not pred_label_add:
                pred_label_add = True
                pred_l = ax.plot(curr_x, pred_y, '-o', color=pred_color, linewidth=1, label='Predict_y')
            else:
                ax.plot(curr_x, pred_y, '-o', color=pred_color, linewidth=1)
    
    true_x = np.sort(list(all_true_tracker.keys()))
    mean_true_y = [np.mean(all_true_tracker[key]) for key in true_x]
    ax.plot(true_x, mean_true_y, '-', color='#fc8d62', linewidth=15, alpha=0.3, label='Mean_True_y')
    max_x = np.max(true_x)
    # ax.set_xlim(right=max_x)
    base_ticks= [2, 4, 8, 16]
    ax.set_xticks([xtick for xtick in base_ticks if xtick <=max_x])
    ax.margins(0.1)           # Default margin is 0.05, value 0 means fit
                
    ax.set_xlabel('Num CPU')
    ax.set_ylabel('Latency')
    
    ax.axis('tight')
    plt.legend(bbox_to_anchor=(1, 0.1, 1, 1), loc='upper left', ncol=1)#, mode="expand")

    plt.savefig(f'figs/prediction/ycs_pred_{method}.pdf', bbox_inches = 'tight')
    plt.show()

In [17]:
'''
Use a dictionary of models
- key: (lower SKU, higher SKU)
- value: model
'''
def predict(model_dicts, X, y_true, expr_idxs, method, plot=True):
    overall_data = pd.DataFrame(zip(expr_idxs, y_true), columns=[expr_label, y_true_label])
    overall_data[X_label] = X

    # for each pair of SKU
    num_cpus = np.sort(np.unique(X))
    test_scores = []
    models, datas_big, datas_small = [], [], []
    dfs = []
    
    for i in range(len(num_cpus)):
        for j in range(i, len(num_cpus)):
            if i == j:
                continue
            cpu_a = num_cpus[i] # smaller
            cpu_b = num_cpus[j] # larger
            
            curr_smaller = overall_data[overall_data[X_label] == cpu_a]
            curr_bigger = overall_data[overall_data[X_label] == cpu_b]
            
            new_preds = []
            
            for model_dict in model_dicts:            
                new_y_true, new_y_pred = [], []

                curr_smaller_pred = model_dict[(cpu_a, cpu_b)].predict(curr_smaller[X_label].to_numpy().reshape(-1, 1))
                curr_bigger_pred = model_dict[(cpu_a, cpu_b)].predict(curr_bigger[X_label].to_numpy().reshape(-1, 1))
                curr_smaller = curr_smaller.assign(Y_PRED=curr_smaller_pred)
                curr_bigger = curr_bigger.assign(Y_PRED=curr_bigger_pred)

                for _, smaller_row in curr_smaller.iterrows():
                    curr_expr_idx = smaller_row[expr_label]
                    curr_diff = smaller_row[y_true_label] - smaller_row[y_pred_label]
                    bigger_row = curr_bigger[curr_bigger[expr_label] == curr_expr_idx]
                    assert(bigger_row.shape[0] == 1)
                    bigger_row = bigger_row.iloc[0]
                    new_y_true.append(bigger_row[y_true_label])
                    new_y_pred.append(bigger_row[y_pred_label] + curr_diff)
                new_preds.append(new_y_pred)
            
            curr_bigger = curr_bigger.assign(Y_PRED=np.mean(np.array(new_preds), axis=0))
            df = pd.merge(curr_smaller, curr_bigger, on=[expr_label], suffixes=suffix_labels)
            dfs.append(df)
            
#             new_y_true = np.array(new_y_true)
#             new_y_pred = np.array(new_y_pred) 
            
#             # score = rmse_score(new_y_true, new_y_pred)/(np.max(new_y_true) - np.min(new_y_true))
#             score = nrmse_score(new_y_true, new_y_pred)
            score = nrmse_score(df['Y_TRUE_large'].to_numpy(), df['Y_PRED_large'].to_numpy())

            test_scores.append(score)
                
            models.append(model_dict[(cpu_a, cpu_b)])
            datas_small.append(curr_smaller)
            datas_big.append(curr_bigger)

    print(test_scores)
    if plot:
        plot_prediction(dfs, X, y_true, expr_idxs, method)
    # print(np.max(y) - np.min(y))
    # overall_score = np.sqrt((np.array(test_scores)**2).mean())/(np.max(y) - np.min(y))
    overall_score = np.mean(test_scores)
    return overall_score

In [18]:
'''
Use a dictionary of models
- key: (lower SKU, higher SKU)
- value: model
'''
def predict_pair(model_dicts, sku_pair, X, y_true, expr_idxs, method, plot=True):
    overall_data = pd.DataFrame(zip(expr_idxs, y_true), columns=[expr_label, y_true_label])
    overall_data[X_label] = X

    # for each pair of SKU
    num_cpus = np.sort(np.unique(X))
    test_scores = []
    models, datas_big, datas_small = [], [], []
    dfs = []
    
    for i in range(len(num_cpus)):
        for j in range(i, len(num_cpus)):
            if i != sku_pair[0] or j != sku_pair[1]:
                continue
            cpu_a = num_cpus[i] # smaller
            cpu_b = num_cpus[j] # larger
            
            curr_smaller = overall_data[overall_data[X_label] == cpu_a]
            curr_bigger = overall_data[overall_data[X_label] == cpu_b]
            
            new_preds = []
            
            for model_dict in model_dicts:            
                new_y_true, new_y_pred = [], []

                curr_smaller_pred = model_dict[(cpu_a, cpu_b)].predict(curr_smaller[X_label].to_numpy().reshape(-1, 1))
                curr_bigger_pred = model_dict[(cpu_a, cpu_b)].predict(curr_bigger[X_label].to_numpy().reshape(-1, 1))
                curr_smaller = curr_smaller.assign(Y_PRED=curr_smaller_pred)
                curr_bigger = curr_bigger.assign(Y_PRED=curr_bigger_pred)

                for _, smaller_row in curr_smaller.iterrows():
                    curr_expr_idx = smaller_row[expr_label]
                    curr_diff = smaller_row[y_true_label] - smaller_row[y_pred_label]
                    bigger_row = curr_bigger[curr_bigger[expr_label] == curr_expr_idx]
                    assert(bigger_row.shape[0] == 1)
                    bigger_row = bigger_row.iloc[0]
                    new_y_true.append(bigger_row[y_true_label])
                    new_y_pred.append(bigger_row[y_pred_label] + curr_diff)
                new_preds.append(new_y_pred)

            curr_bigger = curr_bigger.assign(Y_PRED=np.mean(np.array(new_preds), axis=0))
            df = pd.merge(curr_smaller, curr_bigger, on=[expr_label], suffixes=suffix_labels)
            dfs.append(df)
            
            score = nrmse_score(df['Y_TRUE_large'].to_numpy(), df['Y_PRED_large'].to_numpy())
            test_scores.append(score)
                
            models.append(model_dict[(cpu_a, cpu_b)])
            datas_small.append(curr_smaller)
            datas_big.append(curr_bigger)
            
    print(test_scores)

    if plot:        
        plot_prediction(dfs, X, y_true, expr_idxs, method)
    overall_score = np.mean(test_scores)
    return overall_score

In [19]:
# ycsb_X = get_cpu_nums_as_X(ycsb_data.cpu_nums)
ycsb_X = get_data_X(ycsb_data)
ycsb_y = np.array(ycsb_data.wl_throughput)
ycsb_expr = np.array(ycsb_data.sampled_run_idx)

12


In [20]:
all_results = {}
all_times = {}

## NN

Use only CPU number and target (latency)

In [21]:
from sklearn.neural_network import MLPRegressor
# X_label = 'cpu_num'
y_label = 'latency'

In [22]:
def build_nn_model_indi(X, y, skus, expr_idxs, wl_name, grouping_type, groupping_id, plot=True): 
    train_scores, test_scores = [], []

    run_label = 'RUN_IDX'
    sku_vals = [int(e[3:]) for e in skus]
    overall_data = pd.DataFrame(zip(sku_vals, expr_idxs, y), columns=['CPU_NUM', run_label, y_label])
    overall_data[X_label] = X
    # print(overall_data)
    
    # print(overall_data['CPU_NUM'])
    k = 5
    kf = KFold(n_splits=k, shuffle=True)
    uq_idx = np.unique(expr_idxs)
    num_cpus = np.sort(np.unique(sku_vals))

    models, datas = [], []
    
    model_dict = {}
    train_time = 0

    for i in range(len(num_cpus)):
        for j in range(i, len(num_cpus)):
            if i == j:
                continue
            cpu_a = num_cpus[i] # smaller
            cpu_b = num_cpus[j] # larger
            curr_data = overall_data[ (overall_data['CPU_NUM'] == cpu_a) | (overall_data['CPU_NUM'] == cpu_b) ]
            # print(overall_data['CPU_NUM'])
            # print(overall_data['CPU_NUM'] == cpu_a)
            # print(curr_data)
            datas.append(curr_data)
            start = time.time()
            # select by sampled run idx 
            for train_run_idxs, test_run_idxs in kf.split(uq_idx):
                train_runs = uq_idx[train_run_idxs]
                test_runs = uq_idx[test_run_idxs]
                train_index = [i for i, value in enumerate(curr_data[run_label].to_list()) if value in train_runs]
                test_index = [i for i, value in enumerate(curr_data[run_label].to_list()) if value in test_runs]
                # train_index = curr_data[run_label] in train_runs
                # test_index = curr_data[run_label] in test_runs
                
                data = curr_data.iloc[train_index]
                test = curr_data.iloc[test_index]
                
                model = MLPRegressor(
                    # hidden_layer_sizes=(10,10, 5,),  
                    hidden_layer_sizes=(200, 150, 100, 50, 10, 5),  
                    # hidden_layer_sizes=(10, 5),  
                    activation='relu',
                    solver='adam',
                    # learning_rate='adaptive',
                    random_state=42
                )
                train_x = np.array(data[X_label].to_list())
                test_x = np.array(test[X_label].to_list())
                model.fit(train_x, data[y_label])
                test_pred = model.predict(test_x)
                train_pred = model.predict(train_x)

                score = nrmse_score(test[y_label], test_pred)
                test_scores.append(score)
                
                score = nrmse_score(data[y_label], train_pred)
                train_scores.append(score)
            end = time.time()
            train_time += (end - start) / k
            
            models.append(model)
            model_dict[(cpu_a, cpu_b)] = model

    overall_test = np.mean(test_scores)
    overall_train = np.mean(train_scores)

    return overall_test, overall_train, model_dict, train_time

In [23]:
all_tests, all_trains = [], []
nn_group_to_model_dict = {} 
all_results['NN'] = {}
all_times['NN'] = {}
for ty, curr_data in data_by_type.items():
    name = curr_data.wl_names[0]
    term = curr_data.terminal_num[0]
    if name not in all_results['NN']:
        all_results['NN'][name] = {}    
        all_times['NN'][name] = {}

    print(f'Group Id: {ty}, wl name {name}, terminal num {term}')
    # X = get_cpu_nums_as_X(curr_data.cpu_nums)
    X = get_data_X(curr_data)
    y = np.array(curr_data.wl_throughput)

    test_r2_mean, train_r2_mean, model_dict, train_time = build_nn_model_indi(X, y, curr_data.cpu_nums, np.array(curr_data.sampled_run_idx), name, 'group', curr_data.wl_groups[0], plot=OVERALL_PLOT)
    nn_group_to_model_dict[ty] = model_dict
    print('Test rmse         = {}, Train rmse         = {}'.format(test_r2_mean, train_r2_mean)) 
    all_tests.append(test_r2_mean)
    all_trains.append(train_r2_mean)
    all_results['NN'][name][term] = test_r2_mean
    all_times['NN'][name][term] = train_time

print('Overall test nrmse: {}; train {}'.format(np.mean(all_tests), np.mean(all_trains)))

Group Id: 1, wl name tpcc, terminal num 32
120
Test rmse         = 1.0339189197825396, Train rmse         = 0.8090948249827713
Group Id: 2, wl name tpch, terminal num 1
360
Test rmse         = 2.780864180156225, Train rmse         = 2.053326931233398
Group Id: 3, wl name twitter, terminal num 32
120
Test rmse         = 7.694835777087804, Train rmse         = 5.494851994225933
Group Id: 4, wl name twitter, terminal num 8
120
Test rmse         = 2.4647650627235453, Train rmse         = 1.9646967587373532
Group Id: 7, wl name tpcc, terminal num 8
120
Test rmse         = 0.9129703731141264, Train rmse         = 0.7823197476512648
Group Id: 8, wl name tpcc, terminal num 4
120
Test rmse         = 0.7344229705881057, Train rmse         = 0.56573561525881
Group Id: 9, wl name twitter, terminal num 4
120
Test rmse         = 1.1996583796128537, Train rmse         = 0.9871210936852427
Overall test nrmse: 2.4030622375807424; train 1.808163852253539


In [24]:
terminals = [4, 8, 32]
workloads = ['tpcc', 'twitter', 'tpch']
methods = ['NN']

for me in methods:
    curr_line = ''
    scs = []
    times = []
    for wl in workloads:
        if wl == 'tpch':
            sc = abs(all_results[me][wl][1])
            curr_line += ' \hlrfive{'
            curr_line += f'{sc:.3f}'
            curr_line += '} &'
            scs.append(sc)
            times.append(all_times[me][wl][1])
        else:
            for ter in terminals:
                sc = abs(all_results[me][wl][ter])
                curr_line += ' \hlrfive{'
                curr_line += f'{sc:.3f}'
                curr_line += '} &'
                scs.append(sc)
                times.append(all_times[me][wl][ter])
    curr_line += '\hlrfive{' + f'{np.mean(scs):.3f}'+'}'
    # curr_line += " \\\\ \cline{2-11}"
    curr_line += " \\\\ \hline"
    curr_line = '&\hlrfive{' + f'{me}' + '} & \hlrfive{' + f'{np.mean(times):.4f}' + '} & ' + curr_line
    print(curr_line)

&\hlrfive{NN} & \hlrfive{1.6856} &  \hlrfive{0.734} & \hlrfive{0.913} & \hlrfive{1.034} & \hlrfive{1.200} & \hlrfive{2.465} & \hlrfive{7.695} & \hlrfive{2.781} &\hlrfive{2.403} \\ \hline


In [25]:
for me in methods:
    curr_line = ''
    scs = []
    times = []
    for wl in workloads:
        if wl == 'tpch':
            sc = abs(all_results[me][wl][1])
            curr_line += f' {sc:.3f} &'
            scs.append(sc)
            times.append(all_times[me][wl][1])
        else:
            for ter in terminals:
                sc = abs(all_results[me][wl][ter])
                curr_line += f' {sc:.3f} &'
                scs.append(sc)
                times.append(all_times[me][wl][ter])
    scs = np.array(scs)
    mask = np.logical_or(scs == scs.max(), scs == scs.min())
    
    a_masked = np.ma.masked_array(scs, mask = mask)
    # print(scs, mask)

    curr_line += f' {np.mean(a_masked):.3f}'
    curr_line += " \\\\ \cline{2-11}"
    curr_line = f'& {me} & ' + f' {np.mean(times):.4f} & ' + curr_line
    print(curr_line)

& NN &  1.6856 &  0.734 & 0.913 & 1.034 & 1.200 & 2.465 & 7.695 & 2.781 & 1.678 \\ \cline{2-11}


In [26]:
def build_nn_model(X, y, skus, wl_name, grouping_type, groupping_id, plot=False, show_fig=False):  
    train_rmses, test_rmses = [], []
    k = 5
    sku_vals = [int(e[3:]) for e in skus]
    num_cpus = np.sort(np.unique(sku_vals))
    train_time = 0

    kf = KFold(n_splits=k, shuffle=True)
    for train_index, test_index in kf.split(X):
        start = time.time()
        # X_train = X[train_index]
        X_train = np.array([X[idx] for idx in train_index ])
        y_train = y[train_index]
        # X_test = X[test_index]
        X_test = np.array([X[idx] for idx in test_index])
        y_test = y[test_index]
        
        sku_train = [sku_vals[idx] for idx in train_index]
        sku_test = [sku_vals[idx] for idx in test_index]
        
        model = MLPRegressor(
            # hidden_layer_sizes=(10,10, 5,),  
            hidden_layer_sizes=(200, 150, 100, 50, 10, 5),  
            # hidden_layer_sizes=(10, 5),  
            activation='relu',
            solver='adam',
            # learning_rate='adaptive',
            random_state=42
        )

        model.fit(X_train, y_train)
        test_pred = model.predict(X_test)
        train_pred = model.predict(X_train)
        end = time.time()
        train_time += end - start
        for i in range(len(num_cpus)):
            for j in range(i, len(num_cpus)):
                if i == j:
                    continue
                cpu_a = num_cpus[i] # smaller
                cpu_b = num_cpus[j] # larger
                curr_mask = [x_lab == cpu_a or x_lab == cpu_b for x_lab in sku_test]
                curr_y_true = y_test[curr_mask]
                # rmse = np.sqrt(((curr_y_true-test_pred[curr_mask])**2).mean())
                # n_rmse = rmse / (np.max(curr_y_true)-np.min(curr_y_true))
                n_rmse = nrmse_score(curr_y_true, test_pred[curr_mask])

                test_rmses.append(n_rmse)
                
                curr_mask = [x_lab == cpu_a or x_lab == cpu_b for x_lab in sku_train]
                
                curr_y_true = y_train[curr_mask]
                # rmse = np.sqrt(((curr_y_true-train_pred[curr_mask])**2).mean())
                # n_rmse = rmse / (np.max(curr_y_true)-np.min(curr_y_true))
                n_rmse = nrmse_score(curr_y_true, train_pred[curr_mask])
                train_rmses.append(n_rmse)
    
    train_time /= k

    overall_test_rmse = np.mean(test_rmses)
    overall_train_rmse = np.mean(train_rmses)
    return overall_test_rmse, overall_train_rmse, train_time

In [27]:
all_tests, all_trains = [], []
all_results['nn'] = {}
all_times['nn'] = {}

for ty, curr_data in data_by_type.items():
    name = curr_data.wl_names[0]
    term = curr_data.terminal_num[0]
    if name not in all_results['nn']:
        all_results['nn'][name] = {}
        all_times['nn'][name] = {}
    print(f'Group Id: {ty}, wl name {name}, terminal num {term}')
    
    X = get_data_X(curr_data)
    y = np.array(curr_data.wl_throughput)
    test_r2_mean, train_r2_mean, train_time = build_nn_model(X, y, curr_data.cpu_nums,
                                                                     curr_data.wl_names[0], 'group', 
                                                                     curr_data.wl_groups[0], 
                                                                     plot=OVERALL_PLOT, show_fig=False)
    print('Test rmse = {}, Train rmse = {}'.format(test_r2_mean, train_r2_mean))
    all_tests.append(test_r2_mean)
    all_trains.append(train_r2_mean)
    all_results['nn'][name][term] = test_r2_mean
    all_times['nn'][name][term] = train_time
print('Overall test nrmse: {}, train {}'.format(np.mean(all_tests), np.mean(all_trains)))

Group Id: 1, wl name tpcc, terminal num 32
120
Test rmse = 1.151271851064403, Train rmse = 0.9120665863599642
Group Id: 2, wl name tpch, terminal num 1
360
Test rmse = 1.9711711426301748, Train rmse = 1.5989147860276052
Group Id: 3, wl name twitter, terminal num 32
120
Test rmse = 8.547463251953733, Train rmse = 6.071638251508191
Group Id: 4, wl name twitter, terminal num 8
120
Test rmse = 2.7681984444728753, Train rmse = 2.2308456716258886
Group Id: 7, wl name tpcc, terminal num 8
120
Test rmse = 0.9312639400243554, Train rmse = 0.783652884204318
Group Id: 8, wl name tpcc, terminal num 4
120
Test rmse = 0.6665004307422552, Train rmse = 0.5401193829927874
Group Id: 9, wl name twitter, terminal num 4
120
Test rmse = 1.1848947923087623, Train rmse = 1.0424141831497642
Overall test nrmse: 2.460109121885222, train 1.8828073922669313


In [30]:
terminals = [4, 8, 32]
workloads = ['tpcc', 'twitter', 'tpch']
methods = ['nn']

for me in methods:
    curr_line = ''
    scs = []
    times = []
    for wl in workloads:
        if wl == 'tpch':
            sc = abs(all_results[me][wl][1])
            curr_line += ' \hlrfive{'
            curr_line += f'{sc:.3f}'
            curr_line += '} &'
            scs.append(sc)
            times.append(all_times[me][wl][1])
        else:
            for ter in terminals:
                sc = abs(all_results[me][wl][ter])
                curr_line += ' \hlrfive{'
                curr_line += f'{sc:.3f}'
                curr_line += '} &'
                scs.append(sc)
                times.append(all_times[me][wl][ter])
    curr_line += '\hlrfive{' + f'{np.mean(scs):.3f}'+'}'
    # curr_line += " \\\\ \cline{2-11}"
    curr_line += " \\\\ \hline"
    curr_line = '&\hlrfive{' + f'{me}' + '} & \hlrfive{' + f'{np.mean(times):.4f}' + '} & ' + curr_line
    print(curr_line)

&\hlrfive{nn} & \hlrfive{0.3045} &  \hlrfive{0.667} & \hlrfive{0.931} & \hlrfive{1.151} & \hlrfive{1.185} & \hlrfive{2.768} & \hlrfive{8.547} & \hlrfive{1.971} &\hlrfive{2.460} \\ \hline


In [29]:
for me in methods:
    curr_line = ''
    scs = []
    times = []
    for wl in workloads:
        if wl == 'tpch':
            sc = abs(all_results[me][wl][1])
            curr_line += f' {sc:.3f} &'
            scs.append(sc)
            times.append(all_times[me][wl][1])
        else:
            for ter in terminals:
                sc = abs(all_results[me][wl][ter])
                curr_line += f' {sc:.3f} &'
                scs.append(sc)
                times.append(all_times[me][wl][ter])
    scs = np.array(scs)
    mask = np.logical_or(scs == scs.max(), scs == scs.min())
    
    a_masked = np.ma.masked_array(scs, mask = mask)
    # print(scs, mask)

    curr_line += f' {np.mean(a_masked):.3f}'
    curr_line += " \\\\ \cline{2-11}"
    curr_line = f'& {me} & ' + f' {np.mean(times):.4f} & ' + curr_line
    print(curr_line)

& NN &  1.6856 &  0.734 & 0.913 & 1.034 & 1.200 & 2.465 & 7.695 & 2.781 & 1.678 \\ \cline{2-11}
